In [1]:
from tkinter import *
from tkinter.ttk import Frame, Button, Style

import librosa
import numpy as np
import matplotlib.pyplot as plt
import soundfile as sf
import sounddevice as sd

import argparse
import tempfile
import queue
import sys
import keyboard

In [2]:
import urllib
import json
from bs4 import BeautifulSoup

In [3]:
from vnnews import vnexpress
from vnnews import const

from recorder import recorder

In [4]:
misc = sd.query_devices()
misc

   0 HDA Intel PCH: ALC255 Analog (hw:0,0), ALSA (2 in, 2 out)
   1 HDA Intel PCH: ALC255 Alt Analog (hw:0,2), ALSA (2 in, 0 out)
   2 HDA Intel PCH: HDMI 0 (hw:0,3), ALSA (0 in, 8 out)
   3 HDA Intel PCH: HDMI 1 (hw:0,7), ALSA (0 in, 8 out)
   4 HDA Intel PCH: HDMI 2 (hw:0,8), ALSA (0 in, 8 out)
   5 HDA Intel PCH: HDMI 3 (hw:0,9), ALSA (0 in, 8 out)
   6 sysdefault, ALSA (128 in, 128 out)
   7 front, ALSA (0 in, 2 out)
   8 surround40, ALSA (0 in, 2 out)
   9 surround51, ALSA (0 in, 2 out)
  10 surround71, ALSA (0 in, 2 out)
  11 hdmi, ALSA (0 in, 8 out)
* 12 default, ALSA (128 in, 128 out)
  13 dmix, ALSA (0 in, 2 out)

In [5]:
sd.default.device = 0

In [ ]:
class Example(Frame):
    tp = None
    def __init__(self, parent):
        Frame.__init__(self, parent)
        self.parent = parent
        self.initUI()
        
    def load_news(self, num):
        global tp
        if num == 1:
            content = vnexpress.thoisu()
            tp = "thoisu"
        if num == 2:
            content = vnexpress.gocnhin()
            tp = "gocnhin"
        if num == 3:
            content = vnexpress.thegioi()
            tp = "thegioi"
        if num == 4:
            content = vnexpress.kinhdoanh()
            tp = "kinhdoanh"
        if num == 5:
            content = vnexpress.giaitri()
            tp = "giaitri"
        if num == 6:
            content = vnexpress.thethao()
            tp = "thethao"
        if num == 7:
            content = vnexpress.phapluat()
            tp = "phapluat"
        if num == 8:
            content = vnexpress.giaoduc()
            tp = "giaoduc"
        if num == 9:
            content = vnexpress.suckhoe()
            tp = "suckhoe"
        if num == 10:
            content = vnexpress.doisong()
            tp = "doisong"
        if num == 11:
            content = vnexpress.dulich()
            tp = "dulich"
        if num == 12:
            content = vnexpress.khoahoc()
            tp = "khoahoc"
        if num == 13:
            content = vnexpress.sohoa()
            tp = "sohoa"
        if num == 14:
            content = vnexpress.xe()
            tp = "xe"
        if num == 15:
            content = vnexpress.ykien()
            tp = "ykien"
        if num == 16:
            content = vnexpress.tamsu()
            tp = "tamsu"
        
        text = Text(app, font="Times 12")  
        text.insert(INSERT, content)  
        text.insert(END, "")  
        text.pack() 
        
        textCloseButton = Button(self, text="Close", command=lambda: [text.pack_forget(),
                                                                      textCloseButton.place_forget()])
        textCloseButton.place(x=550, y=400)
        
    def async_playback(self, filename):
        data, sr = sf.read(filename)
        sd.play(data, sr)
        return data, sr
    
    def callback(self, indata, frames, time, status):
        if status:
            print(status, file=sys.stderr)
        q.put(indata.copy())

    def rec(self, filename=None, topic=None, c=0):
        global count
        try:
            count += 1
            if filename is None:
                if topic is None: 
                    filename = tempfile.mktemp(prefix='unknown_sentence_',
                                                    suffix='.wav', dir='')
                else:
                    filename = topic+'_sentence_'+str(count)+'.wav'

            # Make sure the file is opened before recording anything:
            with sf.SoundFile(filename, mode='x', samplerate=48000,
                                channels=1, subtype=None) as file:
                with sd.InputStream(samplerate=48000, device=None,
                                    channels=1, callback=self.callback):
                    print('#' * 80)
                    print('Click "Stop Kernel" on Jupiter to stop recording')
                    print('#' * 80)
                    while True:
                        file.write(q.get())
        except KeyboardInterrupt:
            print('\nRecording finished: ' + repr(filename))
        except Exception as e:
            print(type(e).__name__ + ': ' + str(e))
            
#   Giao dien UI
    def initUI(self):
        global tp
        global count
        tp = None
        count = 0
#       Ten window app
        self.parent.title("News Voice")
        self.style = Style()
        self.style.theme_use("clam")
        self.pack(fill=BOTH, expand=True)
#       Button
        thoisuButton = Button(self, text="Thoi su", command=lambda *args: self.load_news(1))
        thoisuButton.place(x=2, y=0)
        gocnhinButton = Button(self, text="Goc nhin", command=lambda *args: self.load_news(2))
        gocnhinButton.place(x=102, y=0)
        thegioiButton = Button(self, text="The gioi", command=lambda *args: self.load_news(3))
        thegioiButton.place(x=202, y=0)
        kinhdoanhButton = Button(self, text="Kinh doanh", command=lambda *args: self.load_news(4))
        kinhdoanhButton.place(x=302, y=0)
        giaitriButton = Button(self, text="Giai tri", command=lambda *args: self.load_news(5))
        giaitriButton.place(x=402, y=0)
        thethaoButton = Button(self, text="The thao", command=lambda *args: self.load_news(6))
        thethaoButton.place(x=502, y=0)
        phapluatButton = Button(self, text="Phap luat", command=lambda *args: self.load_news(7))
        phapluatButton.place(x=602, y=0)
        giaoducButton = Button(self, text="Giao duc", command=lambda *args: self.load_news(8))
        giaoducButton.place(x=702, y=0)
        suckhoeButton = Button(self, text="Suc khoe", command=lambda *args: self.load_news(9))
        suckhoeButton.place(x=802, y=0)
        doisongButton = Button(self, text="Doi song", command=lambda *args: self.load_news(10))
        doisongButton.place(x=902, y=0)
        dulichButton = Button(self, text="Du lich", command=lambda *args: self.load_news(11))
        dulichButton.place(x=1002, y=0)
        khoahocButton = Button(self, text="Khoa hoc", command=lambda *args: self.load_news(12))
        khoahocButton.place(x=1102, y=0)
        sohoaButton = Button(self, text="So hoa", command=lambda *args: self.load_news(13))
        sohoaButton.place(x=402, y=30)
        xeButton = Button(self, text="Xe", command=lambda *args: self.load_news(14))
        xeButton.place(x=502, y=30)
        ykienButton = Button(self, text="Y kien", command=lambda *args: self.load_news(15))
        ykienButton.place(x=602, y=30)
        tamsuButton = Button(self, text="Tam su", command=lambda *args: self.load_news(16))
        tamsuButton.place(x=702, y=30)

#         recordButton = Button(self, text="Record", command=lambda *args: self.rec(None, self.tp))
#         recordButton.place(x=550, y=520)
        recordButton = Button(self, text="Record", command=lambda *args: self.rec(None, tp, count))
        recordButton.place(x=550, y=520)
        playButton = Button(self, text="Play", command=lambda *args: self.async_playback("./sample_wav/piano.wav"))
        playButton.place(x=550, y=490)
#         stopButton = Button(self, text="Stop", command=self.stop_mic_recording)
#         stopButton.place(x=600, y=520)
        quitButton = Button(self, text="Quit", command=root.destroy)
        quitButton.place(x=550, y=550)
        
root = Tk()
root.geometry("1200x600")
q = queue.Queue()
app = Example(root)
root.mainloop()

################################################################################
Click "Stop Kernel" on Jupiter to stop recording
################################################################################

Recording finished: 'kinhdoanh_sentence_1.wav'
################################################################################
Click "Stop Kernel" on Jupiter to stop recording
################################################################################

Recording finished: 'kinhdoanh_sentence_2.wav'
################################################################################
Click "Stop Kernel" on Jupiter to stop recording
################################################################################

Recording finished: 'suckhoe_sentence_3.wav'
################################################################################
Click "Stop Kernel" on Jupiter to stop recording
################################################################################

Recording fin